### Approaching (Almost) Any NLP Problem on Kaggle


* tfidf
* count features
* logistic regression
* naive bayes
* svm
* xgboost
* grid search
* word vectors
* LSTM
* GRU
* Ensembling


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from tqdm import tqdm
'''
tqdm derives from the Arabic word taqaddum (تقدّم) which can mean “progress,” and is an abbreviation for 
“I love you so much” in Spanish (te quiero demasiado). Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done
'''
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

'''
Truncated Singular Value Decomposition (SVD) is a matrix factorization technique that factors a matrix M into the three matrices U, Σ, and V.
'''
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
train = pd.read_csv('../input/spooky-author-identification/train.zip')
test = pd.read_csv('../input/spooky-author-identification/train.zip')
sample = pd.read_csv('../input/spooky-author-identification/sample_submission.zip')


In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample.head()

In [ ]:
train.shape, test.shape

The problem requires us to predict the author, i.e. EAP, HPL and MWS given the text. In simpler words, text classification with 3 different classes.

For this particular problem, Kaggle has specified multi-class log-loss as evaluation metric. This is implemented in the follow way (taken from: https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)



In [ ]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota


We use the LabelEncoder from scikit-learn to convrt text labels to integers, 0, 1, 2

In [ ]:
lbl_enc = preprocessing.LabelEncoder()


In [ ]:
y = lbl_enc.fit_transform(train.author.values)

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, stratify=y,random_state=42, test_size=0.1, shuffle=True)


In [ ]:
xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape

### Building Basic Models

let's start building our very first model
our very first model is a simple TF-IDF (term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression

In [ ]:
# always start with these features. they work (almost) everytime! 
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')


In [ ]:
# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))


In [ ]:
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [ ]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)

In [ ]:
predictions = clf.predict_proba(xvalid_tfv)

In [ ]:
print("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

And there we go. We have our first model with a multiclass logloss of 0.572 .

But we are greedy and want a better score. Lets look at the same model with a different data.

Instead of using TF-IDF, we can also use word counts as features. This can be done easily using CountVectorizer from scikit-learn.



In [ ]:
ctv = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}',
                     ngram_range=(1,3), stop_words = 'english')


# fitting count vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))


In [ ]:
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

In [ ]:
# fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

In [ ]:
print("logloss: %0.3f" % multiclass_logloss(yvalid, predictions))

Next, let's try a very simple model which was quite famous in ancient times - Naive Bayes.

Let's see what happens when we use naive bayes on these two datasets:



In [ ]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
print("loglos: %0.3f" % multiclass_logloss(yvalid, predictions))

Good performance! but the logistic regression on counts is still better! what happens when we use this model on counts data instead?

Whoa! Seems like old stuff still works good!!!! One more ancient algorithms in the list is SVMs. Some people "love" SVMs. So, we must try SVM on this dataset.

Since SVMs take a lot of time, we will reduce the number of features from the TF-IDF using Singular Value Decomposition before applying SVM.

Also, note that before applying SVMs, we must standardize the data.



In [ ]:
# apply SVD, I chose 120 components. 120-200 components are good enough for SVM model

svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)


# scale the data obtained from SVD. renaming variable to reuse without scaling
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)


In [ ]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


In [ ]:
# Fitting a simple Xgboost on Tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)


clf.fit(xtrain_tfv.tocsc(), ytrain)
# tocsc() Return a copy of this matrix in Compressed Sparse Column format

predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))



In [ ]:
# Fitting a simple xgboost on tf-idf svd Features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print("logloss: %0.3f" % multiclass_logloss(yvalid, predictions))

### use here hyperprameter tuning

### Grid Search
Its a technique for hyperparameter optimization. Not so effective but can give good results if you know the grid you want to use. I specify the parameters that should usually be used in this post: http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/ Please keep in mind that these are the parameters I usually use. There are many other methods of hyperparameter optimization which may or may not be as effective.

In this section, I'll talk about grid search using logistic regression.

Before starting with grid search we need to create a scoring function. This is accomplished using the make_scorer function of scikit-learn.



In [ ]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)
'''
Make a scorer from a performance metric or loss function. 
It takes a score function, such as accuracy_score , mean_squared_error , adjusted_rand_index or 
average_precision and returns a callable that scores an estimator's output.

'''

In [ ]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))


This is an improvement of 8% over the original naive bayes score!

In NLP problems, it's customary to look at word vectors. Word vectors give a lot of insights about the data. Let's dive into that.

